In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.model_selection import cross_validate

import statsmodels.api as sm 
import statsmodels.formula.api as smf

import math

In [3]:
cs_data = pd.read_csv('redoing_cs_data.csv')
cs_back_data = pd.read_csv('cs_back_data.csv')

In [4]:
cs_data.head()

,sent_type,university,sent_id,aligned_to,original_sentence,eng_translation,zh_translation,cs_word_id,first_cs_word_form,first_cs_word_translation,...,surprisal_values,average_surprisal,bilingual_corpus_frequency_negative_log_first_cs_word,bilingual_corpus_frequency_negative_log_first_cs_word_trans,length_first_cs_word_form,dependency_distance,if_it_is_root,if_previous_word_is_punctuation,surprisal_of_previous_word,surprisal_first_cs_word_trans
0,code-switch,CMU,2,CMU_296,rich neighborhood ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 ...,Rich neighborhood is very safe (it belongs to ...,富人 区 ， 非常 安全 （ 这里 属于 富人 区 ） ， 旁边 就 是 公园 ， 附近 还...,1,rich,富人,...,6.100998 1.320391 0.7630151 4.4722 3.20577 2.7...,3.144718,11.319098,10.402807,4,1,0,NaN,NaN,6.100998
1,code-switch,CMU,4,CMU_3006,lease 从 八月 开始 ， 具体 日期 可以 商量 。,Lease will begin in August and the date is neg...,租约 从 八月 开始 ， 具体 日期 可以 商量 。,1,lease,租约,...,6.527233 3.646975 4.193077 1.173965 0.5694761 ...,3.155464,7.348806,10.066335,5,8,0,NaN,NaN,6.527233
2,code-switch,CMU,5,CMU_70,house 一共 有 两 层 + 地下室 。,House has two floors plus basement.,房子 一共 有 两 层 + 地下室 。,1,house,房子,...,5.512905 5.228868 0.5955775 1.38895 2.136289 4...,3.268832,6.694125,7.084991,5,2,0,NaN,NaN,5.512905
3,code-switch,CMU,6,CMU_6777,一共 有 两 个 可以 洗澡 的 bathroom 。,There are two bathroom for bathing.,一共 有 两 个 可以 洗澡 的 浴室 。,8,bathroom,浴室,...,4.509301 0.2903937 1.38895 0.4321387 3.465795 ...,2.642274,8.611048,9.709660,8,6,0,0.0,0.895995,5.210801
4,code-switch,CMU,9,CMU_4373,整个 house 家具 齐全 。,The whole house is fully furnished.,整个 房子 家具 齐全 。,2,house,房子,...,3.732348 4.488641 5.760885 6.166569 0.8237057,4.194430,6.694125,7.084991,5,1,0,0.0,3.732348,4.488641


In [5]:
cs_back_data.head()

,Unnamed: 0,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh
0,0,8.291760,AD,2,7.310274
1,1,6.377483,P,1,5.059746
2,2,12.623917,AD,2,5.992529
3,3,11.908693,NN,2,7.351441
4,4,4.681214,CC,1,3.577562


In [6]:
cs_data_all = cs_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh']]

In [8]:
cs_back_data = cs_back_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh']]

In [9]:
code_switched = [1]* len(cs_data_all) #coding original code-switch as 1, code-switch back as 0

In [10]:
cs_data_all['code_switched'] = code_switched

/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
cs_data_all.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
0,46.615353,NN,2,7.351441,1
1,41.578729,NN,2,7.351441,1
2,37.403903,NN,2,7.039197,1
3,40.402814,NN,2,4.353540,1
4,37.403903,NN,2,7.351441,1


In [12]:
code_switched = [0]* len(cs_back_data) #coding original code-switch as 1, code-switch back as 0

In [13]:
cs_back_data['code_switched'] = code_switched

In [14]:
cs_back_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
0,8.291760,AD,2,7.310274,0
1,6.377483,P,1,5.059746,0
2,12.623917,AD,2,5.992529,0
3,11.908693,NN,2,7.351441,0
4,4.681214,CC,1,3.577562,0


In [15]:
frames = [cs_data_all, cs_back_data]
all_data = pd.concat(frames)
all_data.head()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
0,46.615353,NN,2,7.351441,1
1,41.578729,NN,2,7.351441,1
2,37.403903,NN,2,7.039197,1
3,40.402814,NN,2,4.353540,1
4,37.403903,NN,2,7.351441,1


In [16]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
1123,13.585461,NN,2,7.351441,0
1124,15.820484,NN,2,7.351441,0
1125,13.380050,NN,2,7.351441,0
1126,7.292286,NN,2,6.812994,0
1127,5.134998,VE,1,5.096284,0


In [29]:
as_ints = []
for pos in all_data['cs_word_pos_zh']:
    if pos == 'NN' or pos == 'NR' or pos == 'NT':
        as_ints.append(0)
    elif pos == 'VE' or pos == 'VV':
        as_ints.append(1)
    else:
        as_ints.append(2)

In [30]:
all_data['cs_word_pos_zh'] = as_ints

In [32]:
all_data.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh,code_switched
1123,13.585461,0,2,7.351441,0
1124,15.820484,0,2,7.351441,0
1125,13.380050,0,2,7.351441,0
1126,7.292286,0,2,6.812994,0
1127,5.134998,1,1,5.096284,0


In [33]:
all_data_final = all_data[['word_freq_zh', 'cs_word_pos_zh', 'cs_word_length_zh', 'ngram_surp_zh']]

In [34]:
all_data_final.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh
1123,13.585461,0,2,7.351441
1124,15.820484,0,2,7.351441
1125,13.380050,0,2,7.351441
1126,7.292286,0,2,6.812994
1127,5.134998,1,1,5.096284


In [35]:
cs_word_frequency = np.array(all_data_final['word_freq_zh'])
cs_word_length = np.array(all_data_final['cs_word_length_zh'])
ngram_surp_val = np.array(all_data_final['ngram_surp_zh'])

cs_word_frequency_new = (cs_word_frequency-np.mean(cs_word_frequency))*0.5/np.std(cs_word_frequency)
cs_word_length_new = (cs_word_length-np.mean(cs_word_length))*0.5/np.std(cs_word_length)
ngram_surp_new = (ngram_surp_val-np.mean(ngram_surp_val))*0.5/np.std(ngram_surp_val)

all_data_final['word_freq_zh'] = cs_word_frequency_new
all_data_final['cs_word_length_zh'] = cs_word_length_new
all_data_final['ngram_surp_zh'] = ngram_surp_new

/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/Debasmita/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [36]:
all_data_final.tail()

,word_freq_zh,cs_word_pos_zh,cs_word_length_zh,ngram_surp_zh
1123,-0.386793,0,0.196380,0.442705
1124,-0.313383,0,0.196380,0.442705
1125,-0.393540,0,0.196380,0.442705
1126,-0.593495,0,0.196380,0.226705
1127,-0.664352,1,-0.558972,-0.461960


In [37]:
X = all_data_final
y = all_data['code_switched']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [38]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [39]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       362
           1       1.00      1.00      1.00       420

    accuracy                           1.00       782
   macro avg       1.00      1.00      1.00       782
weighted avg       1.00      1.00      1.00       782



In [42]:
model = smf.ols(formula="code_switched ~ word_freq_zh + C(cs_word_pos_zh) + cs_word_length_zh + ngram_surp_zh",data=all_data).fit() 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          code_switched   R-squared:                       0.967
Model:                            OLS   Adj. R-squared:                  0.967
Method:                 Least Squares   F-statistic:                 1.508e+04
Date:                Tue, 19 Apr 2022   Prob (F-statistic):               0.00
Time:                        16:20:13   Log-Likelihood:                 2562.5
No. Observations:                2604   AIC:                            -5113.
Df Residuals:                    2598   BIC:                            -5078.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0377      0.011     -3.350      0.001      -0.060      -0.016
C(cs_word_pos_zh)[T.1]    -0.0134      0.005     -2.572      0.010      -0.024      -0.003
C(cs_word_pos_zh)[T.2]     0.0377      0.005      8.143      0.000       0.029       0.047
word_freq_zh               0.0337      0.000    247.084      0.000       0.033       0.034
cs_word_length_zh         -0.0864      0.003    -27.873      0.000      -0.092      -0.080
ngram_surp_zh             -0.0178      0.001    -12.122      0.000      -0.021      -0.015
==============================================================================
Omnibus:                      532.799   Durbin-Watson:                   1.883
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1872.846
Skew:                          -0.993   Prob(JB):                         0.00
Kurtosis:                       6.649   Cond. No.                         199.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""